In [1]:
import sys
import os
import urllib
import time

import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

from PIL import Image

from tf_trt_models.classification import download_classification_checkpoint, build_classification_graph

%matplotlib inline

W0905 11:25:26.868213 139929929832256 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0905 11:25:26.869752 139929929832256 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/slim-0.1-py3.6.egg/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0905 11:25:26.916887 139929929832256 deprecation_wrapper.py:119] From /root/.local/lib/python3.6/site-packages/slim-0.1-py3.6.egg/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [2]:
import ipywidgets as widgets

In [4]:
MODELS = ['mobilenet_v1_0p25_128', 'mobilenet_v1_0p5_160', 'mobilenet_v1_1p0_224', 'mobilenet_v2_1p0_224',
          'vgg_16', 'vgg_19', 'inception_v1',
          'inception_v2', 'inception_v3', 'inception_v4', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101',
          'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152']
NUM_CLASSES = [1000, 1001]

w1 = widgets.Dropdown(options=MODELS,
                 value='mobilenet_v1_0p25_128',
                 description='model:',
                 disabled=False,)

w2 = widgets.Dropdown(options=NUM_CLASSES,
                 value=1000,
                 description='num classes:',
                 disabled=False,)
display(w1)
display(w2)

Dropdown(description='model:', options=('mobilenet_v1_0p25_128', 'mobilenet_v1_0p5_160', 'mobilenet_v1_1p0_224…

Dropdown(description='num classes:', options=(1000, 1001), value=1000)

In [10]:
MODEL = w1.value
NUM_CLASSES = w2.value
LABELS_PATH = './data/imagenet_labels_%d.txt' % NUM_CLASSES
IMAGE_PATH = './data/dog-yawning.jpg'

### Download the checkpoint and sample image

In [11]:
checkpoint_path = download_classification_checkpoint(MODEL, 'data')

### Build the frozen graph

In [12]:
frozen_graph, input_names, output_names = build_classification_graph(
    model=MODEL,
    checkpoint=checkpoint_path,
    num_classes=NUM_CLASSES
)
print(input_names, output_names)

W0905 11:29:30.133925 139929929832256 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/layers/python/layers/layers.py:1637: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0905 11:29:33.145380 139929929832256 deprecation.py:323] From /root/.local/lib/python3.6/site-packages/tf_trt_models-0.0-py3.6.egg/tf_trt_models/classification.py:232: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0905 11:29:33.146645 139929929832256 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructi

['input'] ['scores']


### Optimized the graph with TensorRT

In [13]:
converter = trt.TrtGraphConverter(
    input_graph_def=frozen_graph,
    nodes_blacklist=output_names, #output nodes
    max_batch_size=1,
    is_dynamic_op=True,
    max_workspace_size_bytes=trt.DEFAULT_TRT_MAX_WORKSPACE_SIZE_BYTES,
    precision_mode=trt.TrtPrecisionMode.FP16,
    minimum_segment_size=50)
trt_graph = converter.convert()

trt_engine_opts = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("trt_engine_opts = {}".format(trt_engine_opts))

trt_engine_opts = 1


In [14]:
MODEL_DIR = os.path.join('.', 'model')
if tf.gfile.Exists(MODEL_DIR) == False:
    tf.gfile.MkDir(MODEL_DIR)

In [15]:
base_name = os.path.splitext(os.path.basename(checkpoint_path))[0]
save_model_file_name = base_name + '_frozen_fp16.pb'
with open(os.path.join(MODEL_DIR, save_model_file_name), 'wb') as f:
    f.write(trt_graph.SerializeToString())

### copy original model.

In [16]:
tf.reset_default_graph()
frozen_graph, input_names, output_names = build_classification_graph(
    model=MODEL,
    checkpoint=checkpoint_path,
    num_classes=NUM_CLASSES,
    is_remove_relu6=False
)

In [17]:
base_name = os.path.splitext(os.path.basename(checkpoint_path))[0]
save_model_file_name = base_name + '_frozen.pb'
with open(os.path.join(MODEL_DIR, save_model_file_name), 'wb') as f:
    f.write(frozen_graph.SerializeToString())

In [18]:
!ls -al ./model

total 1365308
drwxr-xr-x. 2 root root      4096 Sep  5 11:30 .
drwxr-xr-x. 5 root root      4096 Sep  5 11:30 ..
-rw-r--r--. 1 root root 224206341 Sep  5 11:30 inception_resnet_v2_2016_08_30_frozen.pb
-rw-r--r--. 1 root root 448135998 Sep  5 11:30 inception_resnet_v2_2016_08_30_frozen_fp16.pb
-rw-r--r--. 1 root root 241994428 Sep  5 11:24 resnet_v2_152_frozen.pb
-rw-r--r--. 1 root root 483717761 Sep  5 11:24 resnet_v2_152_frozen_fp16.pb
